In [2]:
# Dependencies
import tweepy
import json
import numpy as np
import pandas as pd
import seaborn as sns
from config import consumer_key, consumer_secret, access_token, access_token_secret
import matplotlib.pyplot as plt


# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

# Import and Initialize Sentiment Analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [8]:
# Target User
target_users = ("APTA_Transit", "BART", "MTA", "CTA", "MARTA", "RTCSNV")

# Tweet Texts
tweet_texts = []
tweet_date = []

#variables for holding sentiments
compound_list = []
positive_list = []
negative_list = []
neutral_list = []
counter_list = []
target_list = []


for target in target_users:
    #Counter
    counter = 1
    # Create a loop to iteratively run API requests
    for x in range(1, 6):

        # Get all tweets from home feed (for each page specified)
        public_tweets = api.user_timeline(target, page=x)

        # Loop through all tweets
        for tweet in public_tweets:

            # Print Tweet
            #print(tweet["text"])

            # Store Tweet in Array
            tweet_texts.append(tweet["text"])

            # Store Tweet Date in Array
            tweet_date.append(tweet["created_at"])

            # Run Vader Analysis on each tweet
            results = analyzer.polarity_scores(tweet["text"])
            compound = results["compound"]
            pos = results["pos"]
            neu = results["neu"]
            neg = results["neg"]

            # Add each value to the appropriate list
            compound_list.append(compound)
            positive_list.append(pos)
            negative_list.append(neg)
            neutral_list.append(neu)
            counter_list.append(counter)
            target_list.append(target)
            # Store the data in dictionary
            sentiment = {
                "User": target_list,
                "Tweet": tweet_texts,
                "Date": tweet_date,
                "Compound": compound_list,
                "Positive": positive_list,
                "Neutral": negative_list,
                "Negative": neutral_list,
                "Tweets Ago": counter_list
            }
            #add to counter
            #print(target)
            counter += 1

#df = pd.DataFrame(sentiment)
sentiment_df = pd.DataFrame(sentiment)
        


In [10]:
sentiment_df.describe

<bound method NDFrame.describe of      Compound                            Date  Negative  Neutral  Positive  \
0      0.8553  Wed Apr 11 21:22:08 +0000 2018     0.516    0.000     0.484   
1      0.0000  Tue Apr 10 21:36:02 +0000 2018     1.000    0.000     0.000   
2      0.7717  Tue Apr 10 13:23:30 +0000 2018     0.675    0.000     0.325   
3      0.0000  Mon Apr 09 21:03:27 +0000 2018     1.000    0.000     0.000   
4      0.0000  Fri Apr 06 21:21:35 +0000 2018     1.000    0.000     0.000   
5      0.4588  Thu Apr 05 18:59:20 +0000 2018     0.870    0.000     0.130   
6      0.7003  Wed Apr 04 20:59:16 +0000 2018     0.734    0.000     0.266   
7      0.4199  Tue Apr 03 16:30:17 +0000 2018     0.823    0.000     0.177   
8      0.0000  Mon Apr 02 21:26:03 +0000 2018     1.000    0.000     0.000   
9      0.2732  Fri Mar 30 19:15:13 +0000 2018     0.884    0.000     0.116   
10     0.3612  Thu Mar 29 21:44:52 +0000 2018     0.872    0.000     0.128   
11     0.0000  Wed Mar 28 20:4

In [7]:
#Using Pandas create dataframes for each media company
apta_sentiment_df = sentiment_df.loc[sentiment_df['User'] == 'APTA']
bart_sentiment_df = sentiment_df.loc[sentiment_df['User'] == 'BART']
mta_sentiment_df = sentiment_df.loc[sentiment_df['User'] == 'MTA']
cta_sentiment_df = sentiment_df.loc[sentiment_df['User'] == 'CTA']
marta_sentiment_df = sentiment_df.loc[sentiment_df['User'] == 'MARTA']
rtcsnv_sentiment_df = sentiment_df.loc[sentiment_df['User'] == 'RTCSNV']

#Average the compound vader analysis for each broadcasting company
apta_compound_avg = (f"{np.mean(apta_sentiment_df['Compound']):.3f}")
bart_compound_avg = (f"{np.mean(bart_sentiment_df['Compound']):.3f}")
mta_compound_avg = (f"{np.mean(mta_sentiment_df['Compound']):.3f}")
cta_compound_avg = (f"{np.mean(cta_sentiment_df['Compound']):.3f}")
marta_compound_avg = (f"{np.mean(marta_sentiment_df['Compound']):.3f}")
rtcsnv_compound_avg = (f"{np.mean(rtcsnv_sentiment_df['Compound']):.3f}")

#create DF with all compounded averages
compound_df = [apta_compound_avg, bart_compound_avg, mta_compound_avg, cta_compound_avg, marta_compound_avg, rtcsnv_compound_avg ]

In [ ]:
# Create a plot  
x_axis = np.arange(len(compund_df))

plt.bar(x_axis, compound_df, color=('red', 'cyan', 'blue', 'yellow', 'green'), alpha=0.5, align="edge")

# Tell matplotlib where we would like to place each of our x axis headers
tick_locations = [value+0.4 for value in x_axis]
plt.xticks(tick_locations, target_users)

# Give our chart some labels and a tile
plt.title("Overall Media Sentiment Based on Twitter (4/10/17)")
plt.xlabel("Media Company")
plt.ylabel("Tweet Populalrity")

# Sets the y limits of the current chart
#plt.ylim([-0.20,1])
# Print our chart to the screen
plt.show()
plt.savefig("Avg_Media Sentiment_Bar_Plot_4-10-18.png")
#Sentiment values left to right
#['0.118', '0.367', '0.002', '0.189', '0.015']